In [5]:
import os
import tensorflow as tf
from tensorflow import contrib
from tensorflow.data import Dataset as Ds
import numpy as np
import pandas as pd
import scipy as sp 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

data  =  pd.read_csv('/Users/dawnstear/desktop/chop_cellpred/data.csv')
data_shuffled = shuffle(data)
X_ = data_shuffled.drop(['Labels','TYPE'],axis=1)
y_ = data_shuffled['Labels'] 
X_train, X_test, y_train, y_test = train_test_split(X_.values,y_.values,test_size=0.2,random_state=144)
 

In [10]:
                                                   
BUFFER = 47
BATCH_SIZE = 30

# Create dataset iterator to avoid using feed_dict() (its very slow)
# TRAIN DATASET
X_train_dataset = Ds.from_tensor_slices((X_train))   
y_train_dataset = Ds.from_tensor_slices((y_train)).map(lambda z: tf.one_hot(z, 10))
train_dataset = Ds.zip((X_train_dataset, y_train_dataset)).shuffle(BUFFER).repeat().batch(BATCH_SIZE)

# Create dataset iterator to avoid using feed_dict() (its very slow)
# TEST DATASET
X_test_dataset = Ds.from_tensor_slices((X_test)) 
y_test_dataset = Ds.from_tensor_slices((y_test)).map(lambda z: tf.one_hot(z, 10))
test_dataset = Ds.zip((X_test_dataset, y_test_dataset)).shuffle(BUFFER).repeat().batch(BATCH_SIZE)


# create general iterator, seamlessly switch bt train data and test data sets
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)

# This will return a tuple where next_element[0] = data, next_element[1] = labels
next_element = iterator.get_next()

# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
testing_init_op = iterator.make_initializer(test_dataset)


In [12]:
def nn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    fc1 = tf.layers.dense(bn, 50)
    fc2 = tf.layers.dense(fc1, 50)
    fc2 = tf.layers.dropout(fc2)
    fc3 = tf.layers.dense(fc2, 10)
    return fc3

# create the neural network model
logits = nn_model(next_element[0])

In [13]:
# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# get accuracy
prediction = tf.argmax(logits, 1)
equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

init_op = tf.global_variables_initializer()

In [14]:
# run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
    # now setup the validation run
    valid_iters = 100
    # re-initialize the iterator, but this time with validation data
    sess.run(testing_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy])
        avg_acc += acc[0]
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,
                                                                                 (avg_acc / valid_iters) * 100))

Epoch: 0, loss: 336.268, training accuracy: 3.33%
Epoch: 50, loss: 15.324, training accuracy: 90.00%
Epoch: 100, loss: 0.893, training accuracy: 96.67%
Epoch: 150, loss: 0.001, training accuracy: 100.00%
Epoch: 200, loss: 0.003, training accuracy: 100.00%
Epoch: 250, loss: 0.001, training accuracy: 100.00%
Epoch: 300, loss: 0.000, training accuracy: 100.00%
Epoch: 350, loss: 0.000, training accuracy: 100.00%
Epoch: 400, loss: 0.000, training accuracy: 100.00%
Epoch: 450, loss: 0.000, training accuracy: 100.00%
Epoch: 500, loss: 0.000, training accuracy: 100.00%
Epoch: 550, loss: 0.000, training accuracy: 100.00%


NameError: name 'validation_init_op' is not defined